In [1]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,RANSACRegressor,TheilSenRegressor
from statsmodels.robust.robust_linear_model import RLM
import statsmodels
import pandas as pd
from tqdm import tqdm
import random
from scipy import optimize
from sklearn.covariance import MinCovDet as MCD
import time

In [2]:
main_size = 500

In [3]:
def Add_Column(df, n = main_size):
    return np.column_stack([np.ones([df.shape[0], 1], dtype=np.int32), df])

In [4]:
ideal_coefs = np.array([5,2])

In [5]:
def MC(df, y):
    sum_mnk = 0
    sum_rs = 0
    sum_ts = 0
    sum_m = 0
    sum_mm = 0
    sum_r = 0
    sum_hbr = 0
    sum_lms = 0
    sum_mmr = 0
    sum_hbrr = 0
    nu = 1000
    for i in tqdm(range(nu)):   
        #МНК
        lr = LinearRegression(fit_intercept=False)
        lr.fit(df_norm, y_norm)
        mnk_coefs = lr.coef_
        #ransac
        ransac = RANSACRegressor(lr)
        ransac.fit(df_norm, y_norm)
        ransac_coefs = ransac.estimator_.coef_
        #Theil-Sen Regressor
        ths = TheilSenRegressor(fit_intercept=False)
        ths.fit(df_norm, y_norm)
        ths_coefs = ths.coef_
        #LMS-оценка
        lms = LMS(df_norm, y_norm)
        # M-оценка
        m_est = RLM(y_norm, df_norm, M = statsmodels.robust.norms.TukeyBiweight()).fit()
        m_est_coefs = m_est.params
        ## ММ-оценка
        mm_est = MM(df_norm, y_norm, lms)
        #R-оценка
        beta = optimize.fmin_bfgs(R, 0, args = (df_norm[:,1], y_norm), disp = 0)[0]
        beta_0 = np.median(y_norm - beta*df_norm[:,1])
        r_est = np.array([beta_0, beta])
        #HBR-оценка
        hbr_beta = optimize.fmin_bfgs(HBR, 0, args = (df_norm, y_norm, lms), disp = 0)
        hbr_est = np.array([np.median(y_norm - hbr_beta * df_norm[:,1]),hbr_beta])
        #HBRR-оценка
        hbrr_beta = optimize.fmin_bfgs(HBR, 0, args = (df_norm, y_norm, ransac_coefs), disp = 0)
        hbrr_est = np.array([np.median(y_norm - hbrr_beta * df_norm[:,1]),hbrr_beta])
        #MMR-оценка
        mmr_est = MM(df_norm, y_norm, ransac_coefs)
        #подсчет
        
        sum_mnk += np.linalg.norm(ideal_coefs - mnk_coefs)/nu
        sum_m += np.linalg.norm(ideal_coefs - m_est_coefs)/nu
        sum_ts += np.linalg.norm(ideal_coefs - ths_coefs)/nu
        sum_rs += np.linalg.norm(ideal_coefs - ransac_coefs)/nu
        sum_mm += np.linalg.norm(ideal_coefs - mm_est)/nu
        sum_hbr += np.linalg.norm(ideal_coefs - hbr_est)/nu
        sum_r += np.linalg.norm(ideal_coefs - r_est)/nu
        sum_lms += np.linalg.norm(ideal_coefs - lms)/nu
        sum_mmr += np.linalg.norm(ideal_coefs - mmr_est)/nu
        sum_hbrr += np.linalg.norm(ideal_coefs - hbrr_est)/nu
        
    return sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

In [6]:
def normal(mean, var, sz = main_size):
    return st.norm.rvs(loc = mean, scale = np.sqrt(var), size = sz)

def unif(a,b):
    return st.uniform.rvs(loc = a, scale = b-a, size = main_size)
    
def tukey(var_1, var_2, bp = 0.1, sz = main_size):
    eps = np.array([])
    for i in range(sz):
        c = st.uniform.rvs()
        if c >= bp:
            eps = np.append(eps, st.norm.rvs(scale = np.sqrt(var_1)))
        else:
            eps = np.append(eps, st.norm.rvs(scale = np.sqrt(var_2)))
    return eps
            
def cauchy(sz = main_size):
    return st.t.rvs(df = 1, size = sz)

In [7]:
def LMS(df, y):
    h = int(df.shape[0]/2 + 1)
    Q = 1000000000
    beta = np.array([1,1],dtype=float)
    for i in range(df.shape[0]):
        for j in range(i):
            if df[:,1][i] != df[:,1][j]:
                beta[1] = (y[i]-y[j])/(df[:,1][i] - df[:,1][j])
                beta[0] = df[:,1][i]*y[j] - df[:,1][j]*y[i]
                med = np.median(np.abs(y - np.sum(beta*df, axis=1)))
                if med < Q:
                    Q = med.copy()
                    ans = beta.copy()
    return ans

In [8]:
df_norm = normal(10,5)
eps = normal(0,1)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)
LMS(df_norm, y_norm)

array([4.09133941, 2.10194841])

In [9]:
def MM(df, y, blms):
    n = df.shape[0]
    s = y - np.sum(blms * df, axis = 1)
    res_itog = np.abs(np.median(s) - s)
    MADN = np.median(np.abs(res_itog)) / 0.675
    const1 = 10
    const2 = 4.68
    var = s/(const1*MADN)
    w = np.minimum(3 - 3*np.power(var,2) + np.power(var,4), 1/np.power(var,2))
    sigma_1 = np.dot(w,np.power(s/const1,2))
    sigma_1 = np.sqrt(sigma_1 * 2/ df.shape[0])
    eps = 0.0001
    while(np.abs(sigma_1/MADN - 1) > eps):
        MADN = sigma_1
        sigma_1 = 0
        w = np.minimum(3 - 3*np.power(var,2) + np.power(var,4), 1/np.power(var,2))
        sigma_1 = np.dot(w,np.power(s/const1,2))
        sigma_1 = np.sqrt(sigma_1 * 2 / n)
    var2 = s/(const2*MADN)
    w = np.minimum(3 - 3*np.power(var2,2) + np.power(var2,4), 1/np.power(var2,2))
    W = np.diag(w)
    beta = blms
    beta_1 = np.dot(np.dot(np.dot(np.linalg.inv(np.dot(np.dot(df.T,W), df)), df.T), W), y)
    eps = 0.001
    while np.linalg.norm(beta_1-beta)/np.linalg.norm(beta) > eps:
        s = y - np.sum(beta_1 * df, axis = 1)
        var2 = s/(const2*sigma_1)
        w = np.minimum(3 - 3*np.power(var2,2) + np.power(var2,4), 1/np.power(var2,2))
        W = np.diag(w)
        beta = beta_1
        beta_1 = np.dot(np.dot(np.dot(np.linalg.inv(np.dot(np.dot(df.T,W), df)), df.T), W), y)
    return beta_1

In [10]:
def R(x, df, y):
    s = 0
    array = y - x*df
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    return np.dot(array, ranks/(df.shape[0]) - 0.5)

In [11]:
def HBR(x, df, y, lts):
    resid = y - np.sum(lts * df,axis = 1)
    MADN = 1.483*np.median(np.abs(resid - np.median(resid)))
    mcd = MCD()
    mcd.fit(df[:,1].reshape(-1,1))
    mahal =  mcd.mahalanobis(df[:,1].reshape(-1,1))
    b, c = 4, 4 
    resid = np.abs(resid / MADN)
    resid_new = y - x * df[:,1]
    resid_dif = np.abs(resid_new - resid_new.reshape(-1,1))
    resid_dif = np.tril(resid_dif, -1)
    mahal = b / mahal
    mahal[mahal > 1] = 1
    mahal = mahal / resid
    mahal = mahal.reshape(-1, 1) * mahal
    mahal = np.tril(mahal, -1)
    mahal = c * mahal
    mahal[mahal > 1] = 1
    ans = np.sum(mahal * resid_dif)
    return ans

In [12]:
start_time = time.time()
lms = LMS(df_norm, y_norm)
hbr_beta = optimize.fmin_bfgs(HBR, 0, args = (df_norm, y_norm, lms), disp = 0)
np.array([np.median(y_norm - hbr_beta * df_norm[:,1]),hbr_beta])
print("--- %s out seconds ---" % (time.time() - start_time))

--- 12.649689197540283 out seconds ---


## $X_1 \sim N(20;2), X_2 \sim N(10;3), \varepsilon \sim N(0;3)$

In [112]:
norm1 = normal(20,2)
norm2 = normal(10,3)
df_norm = Add_Column(np.concatenate((norm1, norm2)).reshape(2, main_size).transpose())
eps = normal(0,3)
y_norm = 5 + 2*df_norm[:,1] + df_norm[:, 2] + eps

In [113]:
lr = LinearRegression(fit_intercept=False)
lr.fit(df_norm, y_norm)
mnk_coefs = lr.coef_

In [114]:
ransac = RANSACRegressor(lr)
ransac.fit(df_norm, y_norm)
ransac_coefs = ransac.estimator_.coef_

In [115]:
ths = TheilSenRegressor(fit_intercept=False)
ths.fit(df_norm, y_norm)
ths_coefs = ths.coef_

# Нет кластера выбросов

#  $y = 5 + 2X + \varepsilon$

### $X \sim N(20;2), \varepsilon \sim N(0;3)$

In [116]:
df_norm = normal(20, 2)
eps = normal(0, 3)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts,sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 1000/1000 [3:39:56<00:00, 11.90s/it] 


In [117]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts,sum_mmr, sum_hbrr

(1.2395443848905119,
 1.2776747190232212,
 1.1445692911337324,
 2.3288160962028024,
 1.3326279975945645,
 2.8435727189104396,
 2.512063146598734,
 1.244993800990177,
 1.1327206083010966,
 1.3067205222781608)

In [118]:
sum_mm, sum_hbr, sum_mmr, sum_hbrr

(1.1445692911337324,
 1.3326279975945645,
 1.1327206083010966,
 1.3067205222781608)

### $X \sim N(20;2), \varepsilon \sim CN(0.1;1;100)$

In [95]:
df_norm = normal(20, 2)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [00:03<00:00, 156.35it/s]


In [26]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

(0.2206359959000406,
 0.07178094370980538,
 0.1856164666706461,
 2.593897014142656,
 0.06326606212690097,
 0.2822738749859821,
 0.5302501870277698,
 0.21059537781758358,
 0.17441328606464945,
 0.06086922112221068)

### $X \sim N(10;5), \varepsilon \sim C(0;1)$,

In [97]:
eps = cauchy()
y_norm = 5 + 2*df_norm[:,1] + eps
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [00:04<00:00, 123.17it/s]


In [34]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

(4.427721112599392,
 0.6655974437426757,
 0.6682404994270461,
 2.4679441307588315,
 0.7599884394278767,
 2.487571600651895,
 1.5553655198286995,
 1.6745786018199076,
 0.6867718997584477,
 0.7964747964994527)

### $X \sim U(3;7), \varepsilon \sim N(0;3)$

In [35]:
df_norm = unif(3,7)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)

In [36]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [45:05<00:00,  5.32s/it]


In [37]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

(0.06231814326425268,
 0.05921299436461351,
 0.09953474718062996,
 0.4692082943631697,
 0.09576749686637981,
 0.6750395823265694,
 0.7633215984368703,
 0.08476808950216597,
 0.09608444562036766,
 0.0953637996871203)

### $X \sim U(3;7), \varepsilon \sim C(0;1)$

In [38]:
eps = cauchy()
y_norm = 5 + 2*df_norm[:,1] + eps

In [39]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [47:23<00:00,  5.61s/it]


In [40]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

(32.386542147331745,
 0.2953288224012695,
 0.16017465051336852,
 0.2811166227910118,
 0.18090226001833656,
 0.8397850782844875,
 0.7900480412539894,
 0.15994415875303197,
 0.16698668784017107,
 0.19008623333644342)

### $X \sim CN(0.1;5; 200), \varepsilon \sim N(0; 3)$

In [41]:
df_norm = tukey(5,200)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)

In [42]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [45:35<00:00,  6.21s/it]


In [43]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(0.19351441093090505,
 0.19041722746257916,
 0.03364719437133429,
 0.1388362869603805,
 0.20933535885898927,
 0.20777592531745828,
 0.12313630561145401,
 0.12783718337146932,
 0.1253106911286492,
 0.10411111319574119)

### $X \sim CN(0.1;5;200), \varepsilon \sim CN(0.1;1;100)$

In [44]:
eps = tukey(1,100)
y_norm = 5 + 2*df_norm[:,1] + eps

In [45]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [48:34<00:00,  5.30s/it]


In [46]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(0.17528366979185886,
 0.014876885785761131,
 0.028069562655436125,
 0.23631685885929235,
 0.03576211022211335,
 0.05125771380472158,
 0.029531609626965907,
 0.025741071475429034,
 0.027603595538798483,
 0.021627839558902826)

### $X \sim CN(0.1;5;200), \varepsilon \sim C(0;1)$

In [47]:
eps = cauchy()
y_norm = 5 + 2*df_norm[:,1] + eps

In [48]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [37:22<00:00,  4.55s/it]


In [49]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(0.055950029544546434,
 0.11279937868924388,
 0.16583850123094215,
 0.22223454202060985,
 0.1253593665311841,
 0.11129459795796842,
 0.12561064674202757,
 0.154417973724294,
 0.12698446290204637,
 0.1666242760492171)

### $X \sim C(0;1), \varepsilon \sim N(0;3)$

In [50]:
df_norm = cauchy()
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)

In [51]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [44:58<00:00,  5.24s/it]


In [52]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(0.044892076538561455,
 0.05591302397976471,
 0.02369699689085429,
 0.21467524466619825,
 0.2504729203245886,
 0.05694631179309619,
 0.006910844522891017,
 0.019522309844703097,
 0.01429579397256957,
 0.03192059866558705)

### $X \sim C(0;1), \varepsilon \sim CN(0.1;1;100)$

In [53]:
eps = tukey(1,100)
y_norm = 5 + 2*df_norm[:,1] + eps

In [54]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [44:40<00:00,  4.92s/it]


In [55]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(0.17872860982824554,
 0.058489420025307125,
 0.06820733329102438,
 0.038908039324493576,
 0.07909352609206038,
 0.07867262170391781,
 0.05078471805513082,
 0.09447965797884174,
 0.05185237577710511,
 0.10605898082806735)

### $X \sim C(0;1), \varepsilon \sim C(0;1)$

In [56]:
eps = cauchy()
y_norm = 5 + 2*df_norm[:,1] + eps

In [57]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

100%|██████████| 500/500 [44:55<00:00,  5.30s/it]


In [58]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(2.8815407605926757,
 0.010051254985767717,
 0.044486146367928955,
 0.1922920107460108,
 0.2077478016572025,
 0.12085382727435175,
 0.01690667150172821,
 0.028595513164609083,
 0.01724627365615942,
 0.03486570534066682)

# Кластер выбросов

### $X \sim N(10;5), \varepsilon \sim N(0;3)$

In [102]:
df_norm = normal(10,5)
eps = normal(0,3)
a = np.arange(main_size)
random.shuffle(a)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 1000
df_norm = Add_Column(df_norm)

In [103]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [28:47<16:54,  5.31s/it]


100%|██████████| 500/500 [45:44<00:00,  5.22s/it]


In [104]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts,sum_mmr, sum_hbrr

(181.2178480726731,
 1.2120490268359911,
 1.2105967084182652,
 0.19445649954052013,
 0.8252854733239058,
 1.433921770695133,
 1.2274647286203306,
 0.5022000600945553,
 1.2098539743367176,
 0.8181280296000107)

In [105]:
sum_mm, sum_hbr, sum_mmr, sum_hbrr

(1.2105967084182652,
 0.8252854733239058,
 1.2098539743367176,
 0.8181280296000107)

### $X \sim N(10;5), \varepsilon \sim CN(0.1;1;100)$

In [106]:
df_norm = normal(10,5)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 5000
df_norm = Add_Column(df_norm)

In [107]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [28:45<16:19,  5.13s/it]


100%|██████████| 500/500 [46:36<00:00,  5.10s/it]


In [108]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(515.2810542390685,
 0.04898394423961944,
 0.32258203845395234,
 0.7180436679747372,
 0.12798753940760826,
 0.1096286758183637,
 0.09689174609797903,
 0.1269062029062935,
 0.09918339886755753,
 0.1058947783275676)

### $X \sim N(10;5), \varepsilon \sim C(0;1)$

In [109]:
df_norm = normal(10,5)
eps = cauchy()
a = np.arange(main_size)
random.shuffle(a)
eps[a[:50]] += 5000
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)

In [110]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [26:16<16:24,  5.16s/it]


100%|██████████| 500/500 [43:18<00:00,  5.61s/it]


In [111]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(1023.811894648876,
 0.16573532738882482,
 0.6549013367206382,
 0.0804852481224172,
 0.4356800478719198,
 0.435730169569583,
 0.029773119949389287,
 0.14328273860673968,
 0.029980515542111605,
 0.14429429407334488)

### $X \sim U(3;7), \varepsilon \sim N(0;3)$

In [112]:
df_norm = unif(3,7)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 10000
df_norm = Add_Column(df_norm)

In [113]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [28:56<31:41,  9.96s/it]


100%|██████████| 500/500 [45:13<00:00,  5.01s/it]


In [114]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(1988.8700435060123,
 0.06351970917316327,
 1.2039006514085067,
 0.10254354701779571,
 0.7825341995548722,
 0.7217045966941165,
 0.05985935651904511,
 0.4786289228273802,
 0.06054877435737192,
 0.48154995622318725)

### $X \sim U(3;7), \varepsilon \sim CN(0.1;1;100)$

In [115]:
df_norm = unif(3,7)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 5000
df_norm = Add_Column(df_norm)

In [116]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [23:16<14:26,  4.54s/it]


100%|██████████| 500/500 [37:41<00:00,  4.28s/it]


In [117]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(1751.2932715568138,
 0.0559912636942663,
 1.2877496718083357,
 0.45141769465056897,
 0.17251324473177582,
 0.977571775648633,
 0.04080260852499354,
 0.28703894852808176,
 0.03880197256781791,
 0.2914197194507797)

### $X \sim U(3;7), \varepsilon \sim C(0;1)$

In [118]:
df_norm = unif(3,7)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
y_norm[250:] += 5000
df_norm = Add_Column(df_norm)

In [119]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [24:22<15:20,  4.82s/it]


100%|██████████| 500/500 [39:24<00:00,  4.71s/it]


In [120]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(628.2918627537084,
 0.27205126361660575,
 0.554564116404531,
 0.7766517402249405,
 0.512422094602347,
 0.18348539456610558,
 0.02514551644201835,
 0.26689512965148615,
 0.029646663948429793,
 0.2455160305208193)

### $X \sim CN(0.1;5; 200), \varepsilon \sim N(0; 3)$

In [121]:
df_norm = tukey(5,200)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
y_norm[250:] += 5000
df_norm = Add_Column(df_norm)

In [122]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [25:33<15:34,  4.89s/it]


100%|██████████| 500/500 [41:21<00:00,  5.00s/it]


In [123]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(833.6194092977228,
 0.05534704311114739,
 0.45464190398563625,
 0.21583320158859265,
 0.08470116231383513,
 0.6427099836979181,
 0.055329685470788356,
 0.4661734014076436,
 0.05579664881476692,
 0.45502821953415723)

### $X \sim CN(0.1;5;200), \varepsilon \sim CN(0.1;1;100)$

In [124]:
df_norm = tukey(5,200)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
y_norm[250:] += 10000
df_norm = Add_Column(df_norm)

In [125]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [25:09<15:54,  5.00s/it]


100%|██████████| 500/500 [40:38<00:00,  5.08s/it]


In [126]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(1666.7787274430752,
 0.06939264674653572,
 0.3242321113424621,
 0.27811006173395886,
 0.10912878178836537,
 0.36021655250115864,
 0.05851261806338186,
 0.32162065890236924,
 0.058848670776662323,
 0.3213173803097748)

### $X \sim CN(0.1;5;200), \varepsilon \sim C(0;1)$

In [127]:
df_norm = tukey(5,200)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
y_norm[250:] += 5000
df_norm = Add_Column(df_norm)

In [128]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [22:24<14:01,  4.41s/it]


100%|██████████| 500/500 [36:16<00:00,  4.30s/it]


In [129]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(838.2226482010528,
 0.08348519222782448,
 0.42547656603558914,
 0.0726782037730774,
 0.16082434552279679,
 0.7250791886212891,
 0.11337148906485364,
 0.4330079414813717,
 0.11443412841421892,
 0.4345194654112428)

### $X \sim C(0;1), \varepsilon \sim N(0;3)$

In [130]:
df_norm = cauchy()
eps = normal(0,3)
a = np.arange(100)
random.shuffle(a)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 10000
df_norm = Add_Column(df_norm)

In [131]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [25:33<15:16,  4.80s/it]


100%|██████████| 500/500 [41:31<00:00,  5.05s/it]


In [132]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(1658.5751433392143,
 0.06770308989384445,
 0.3323364435829283,
 0.24948170277794346,
 0.18438718130943527,
 0.40121327535010315,
 0.06943985196811622,
 0.3657317981022015,
 0.06866584188824851,
 0.35120811893728926)

### $X \sim C(0;1), \varepsilon \sim C(0;1)$

In [133]:
df_norm = cauchy()
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 5000
df_norm = Add_Column(df_norm)

In [134]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [21:44<13:30,  4.24s/it]


100%|██████████| 500/500 [35:16<00:00,  4.15s/it]


In [135]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(835.0729326991726,
 0.022750729851291864,
 0.3210088302480844,
 0.0844961938603036,
 0.14952489307595201,
 0.5155000300897653,
 0.03263506476056672,
 0.298737245971406,
 0.03387878841519205,
 0.27952501111161804)

# Засорение иксов

In [305]:
df_norm = normal(10,5)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [306]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 1/500 [00:06<56:40,  6.81s/it]

  0%|          | 2/500 [00:11<51:53,  6.25s/it]

  1%|          | 3/500 [00:18<54:10,  6.54s/it]

  1%|          | 4/500 [00:25<54:24,  6.58s/it]

  1%|          | 5/500 [00:32<53:52,  6.53s/it]

  1%|          | 6/500 [00:38<52:24,  6.36s/it]

  1%|▏         | 7/500 [00:44<53:39,  6.53s/it]

  2%|▏         | 8/500 [00:52<55:23,  6.76s/it]

  2%|▏         | 9/500 [00:58<55:18,  6.76s/it]

  2%|▏         | 10/500 [01:06<56:15,  6.89s/it]

  2%|▏         | 11/500 [01:12<54:10,  6.65s/it]

  2%|▏         | 12/500 [01:17<49:39,  6.10s/it]

  3%|▎         | 13/500 [01:22<48:16,  5.95s/it]

  3%|▎         | 14/500 [01:28<47:09,  5.82s/it]

  3%|▎         | 15/500 [01:34<47:58,  5.93s/it]

  3%|▎         | 16/500 [01:39<46:54,  5.82s/it]

  3%|▎         | 17/500 [01:46<47:58,  5.96s/it]

  4%|▎         | 18/500 [01:52<48:49,  6.08s/it]

  4%|▍         | 19/500 [01:58<48:58,  6.11s/it]

  4%|▍         |

 58%|█████▊    | 288/500 [29:58<20:23,  5.77s/it]

 58%|█████▊    | 289/500 [30:03<19:42,  5.61s/it]

 58%|█████▊    | 290/500 [30:09<19:39,  5.61s/it]

 58%|█████▊    | 291/500 [30:15<19:31,  5.61s/it]

 58%|█████▊    | 292/500 [30:20<19:24,  5.60s/it]

 59%|█████▊    | 293/500 [30:26<19:12,  5.57s/it]

 59%|█████▉    | 294/500 [30:31<19:01,  5.54s/it]

 59%|█████▉    | 295/500 [30:37<18:41,  5.47s/it]

 59%|█████▉    | 296/500 [30:42<18:47,  5.53s/it]

 59%|█████▉    | 297/500 [30:48<18:35,  5.50s/it]

 60%|█████▉    | 298/500 [30:53<18:41,  5.55s/it]

 60%|█████▉    | 299/500 [30:59<18:47,  5.61s/it]

 60%|██████    | 300/500 [31:05<18:40,  5.60s/it]

 60%|██████    | 301/500 [31:10<18:36,  5.61s/it]

 60%|██████    | 302/500 [31:16<18:21,  5.56s/it]

 61%|██████    | 303/500 [31:22<18:32,  5.65s/it]

 61%|██████    | 304/500 [31:27<17:51,  5.47s/it]

 61%|██████    | 305/500 [31:32<17:16,  5.32s/it]

 61%|██████    | 306/500 [31:37<17:28,  5.40s/it]

 61%|██████▏   | 307/500 [31:43

In [307]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts,sum_mmr, sum_hbrr

(19.92232960335195,
 20.022362870048447,
 20.111596477303273,
 20.07607215449277,
 0.6174852360732399,
 1.1784233655383087,
 0.9323323441260465,
 4.946093067378302,
 20.101165640821797,
 0.6006215619748271)

In [308]:
sum_mm, sum_hbr, sum_mmr, sum_hbrr

(20.111596477303273,
 0.6174852360732399,
 20.101165640821797,
 0.6006215619748271)

# ОТСЮДА

### $X \sim N(10;5), \varepsilon \sim CN(0.1;1;100)$

In [309]:
df_norm = normal(10,5)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 1/500 [00:05<49:13,  5.92s/it]

  0%|          | 2/500 [00:12<51:58,  6.26s/it]

  1%|          | 3/500 [00:19<53:06,  6.41s/it]

  1%|          | 4/500 [00:26<53:24,  6.46s/it]

  1%|          | 5/500 [00:32<51:47,  6.28s/it]

  1%|          | 6/500 [00:36<47:58,  5.83s/it]

  1%|▏         | 7/500 [00:43<49:51,  6.07s/it]

  2%|▏         | 8/500 [00:50<52:22,  6.39s/it]

  2%|▏         | 9/500 [00:59<58:08,  7.10s/it]

  2%|▏         | 10/500 [01:09<1:04:39,  7.92s/it]

  2%|▏         | 11/500 [01:16<1:02:36,  7.68s/it]

  2%|▏         | 12/500 [01:24<1:02:22,  7.67s/it]

  3%|▎         | 13/500 [01:31<1:02:09,  7.66s/it]

  3%|▎         | 14/500 [01:38<59:52,  7.39s/it]  

  3%|▎         | 15/500 [01:44<56:21,  6.97s/it]

  3%|▎         | 16/500 [01:49<52:06,  6.46s/it]

  3%|▎         | 17/500 [01:55<50:06,  6.22s/it]

  4%|▎         | 18/500 [02:00<47:40,  5.93s/it]

  4%|▍         | 19/500 [02:07<49:27,  6.17s/it]

  4%|▍

 58%|█████▊    | 288/500 [27:54<18:43,  5.30s/it]

 58%|█████▊    | 289/500 [27:59<18:25,  5.24s/it]

 58%|█████▊    | 290/500 [28:04<18:09,  5.19s/it]

 58%|█████▊    | 291/500 [28:09<17:57,  5.16s/it]

 58%|█████▊    | 292/500 [28:14<17:37,  5.08s/it]

 59%|█████▊    | 293/500 [28:19<17:14,  5.00s/it]

 59%|█████▉    | 294/500 [28:23<16:46,  4.89s/it]

 59%|█████▉    | 295/500 [28:28<16:31,  4.84s/it]

 59%|█████▉    | 296/500 [28:33<16:33,  4.87s/it]

 59%|█████▉    | 297/500 [28:39<17:11,  5.08s/it]

 60%|█████▉    | 298/500 [28:44<17:08,  5.09s/it]

 60%|█████▉    | 299/500 [28:49<17:02,  5.08s/it]

 60%|██████    | 300/500 [28:54<16:57,  5.09s/it]

 60%|██████    | 301/500 [28:59<17:12,  5.19s/it]

 60%|██████    | 302/500 [29:05<17:29,  5.30s/it]

 61%|██████    | 303/500 [29:10<17:30,  5.33s/it]

 61%|██████    | 304/500 [29:16<17:50,  5.46s/it]

 61%|██████    | 305/500 [29:22<17:50,  5.49s/it]

 61%|██████    | 306/500 [29:27<17:56,  5.55s/it]

 61%|██████▏   | 307/500 [29:33

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim N(10;5), \varepsilon \sim C(0;1)$

In [ ]:
df_norm = normal(10,5)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
df_norm[:30] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim N(0;3)$

In [ ]:
df_norm = unif(3,7)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm[:30] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
df_norm = unif(3,7)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
df_norm[:30] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim C(0;1)$

In [ ]:
df_norm = unif(3,7)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5; 200), \varepsilon \sim N(0; 3)$

In [ ]:
df_norm = tukey(5,200)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5;200), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
df_norm = tukey(5,200)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

# Выше моделировалось в ночь!

### $X \sim CN(0.1;5;200), \varepsilon \sim C(0;1)$

In [127]:
df_norm = tukey(5,200)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [128]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)


 62%|██████▏   | 309/500 [22:24<14:01,  4.41s/it]


100%|██████████| 500/500 [36:16<00:00,  4.30s/it]


In [129]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

(838.2226482010528,
 0.08348519222782448,
 0.42547656603558914,
 0.0726782037730774,
 0.16082434552279679,
 0.7250791886212891,
 0.11337148906485364,
 0.4330079414813717,
 0.11443412841421892,
 0.4345194654112428)